In [1]:
# basic
import os
import re
import time
import numpy as np 
import pandas as pd 

# bs4
from bs4 import element 
from bs4 import BeautifulSoup

# selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait


In [12]:

class yahoo_financial_statement():

    """

    fetch data from yahoo finacne 

    3 main financial statement 

    --> "Cash_Flow" , "Income_Statement" , "Balance_Sheet"

    """

    def __init__(self,ticker,selenium_path,fetch_data ):

        self.ticker = ticker 
        self.selenium_path = selenium_path
        self.fetch_data    = fetch_data
        
        url_typ = {
        "Cash_Flow"         : "cash-flow?p=" ,
        "Income_Statement"  : "financials?p=" ,
        "Balance_Sheet"     : "balance-sheet?p=",
        }

        self. url   = 'https://finance.yahoo.com/quote/'+str(self.ticker)+"/"+url_typ[str(self.fetch_data)]+str(self.ticker)
    
    def Web_Driver(self):
        
        # chrome driver setting
        options = webdriver.ChromeOptions()  
        options.add_argument("--incognito")
        options.add_argument("--start-maximized") 
        driver = webdriver.Chrome(self.selenium_path,options=options)
        driver.get(self.url)
        #----------------------
        driver.find_element_by_xpath("//button[@class='expandPf Fz(s) Bd(0) C($linkColor) C($linkActiveColor):h Fw(500) D(n)--print Fl(end) Mt(5px)']").click()
        self.href = driver.page_source

    def get_table_data(self):

        output_df  = pd.DataFrame()
        soup       = BeautifulSoup(self.href , 'html.parser')


        # ----- columns 
        Columns  = soup.find(class_='D(tbhg)')
        column  = Columns.find_all('span')
        column = [cot.text for cot in column]

        # ---- data
        First_Row_Data   = soup.find_all(class_='rw-expnded')
        time.sleep(1)
        偶數_Row_Data     = soup.find_all('div', attrs={'class':'Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg D(tbc)'})
        基數_Row_Data     = soup.find_all('div',  attrs={'class':'Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(100px)--pnclg Bgc($lv1BgColor) fi-row:h_Bgc($hoverBgColor) D(tbc)'})


        # get text data
        First_Row_Data  = [data.find('span').text for data in First_Row_Data]
        Second_Row_Data = [data.text for data in 偶數_Row_Data]
        Third_Row_Data  = [data.text for data in 基數_Row_Data]

        # -----------------------------

        if self.fetch_data == "Balance_Sheet" :

            columns_2 = []
            columns_4 = []

            for i in range(0,len(Second_Row_Data),2) :

                Second_columns_data = Second_Row_Data[i]
                Forth_columns_data  = Second_Row_Data[i+1]

                columns_2 .append(Second_columns_data)
                columns_4 .append(Forth_columns_data)
            
            columns_3 = []
            columns_5 = []

            for i in range(0,len(Third_Row_Data ),2) :

                Third_columns_data = Third_Row_Data[i]
                Fifth_columns_data = Third_Row_Data[i+1]

                columns_3 .append(Third_columns_data)
                columns_5 .append(Fifth_columns_data)

            output_df[column[0]] = First_Row_Data
            output_df[column[1]] = columns_2
            output_df[column[2]] = columns_3
            output_df[column[3]] = columns_4
            output_df[column[4]] = columns_5

            return output_df

        else:
            # Incone Statement & Cash Flow ( 偶數排資料 )

            columns_2 = []
            columns_4 = []

            for i in range(0,len(Second_Row_Data),2) :

                Second_columns_data = Second_Row_Data[i]
                Forth_columns_data  = Second_Row_Data[i+1]

                columns_2 .append(Second_columns_data)
                columns_4 .append(Forth_columns_data)

            # # 基數排資料
            columns_1 = []
            columns_3 = []
            columns_5 = []

            for i in range(0,len(Third_Row_Data ),3) :
                
                First_columns_data = Third_Row_Data[i]
                Third_columns_data = Third_Row_Data[i+1]
                Fifth_columns_data = Third_Row_Data[i+2]

                columns_1 .append(First_columns_data)
                columns_3 .append(Third_columns_data)
                columns_5 .append(Fifth_columns_data)

                

            output_df[column[0]] = First_Row_Data
            output_df[column[1]] = columns_1

            output_df[column[2]] = columns_2
            output_df[column[3]] = columns_3
            output_df[column[4]] = columns_4
            output_df[column[5]] = columns_5

            return output_df


#  "Cash_Flow" , "Income_Statement" , "Balance_Sheet"

In [13]:
Financial_Statement          = yahoo_financial_statement(ticker='MMM',selenium_path=r'/Users/chen-lichiang/Desktop/python/Factor_Analysis/chromedriver',fetch_data='Income_Statement')
Company_Financial_Statement  = Financial_Statement.Web_Driver()
Company_Financial_Statement  = Financial_Statement.get_table_data()
    

Company_Financial_Statement

,Breakdown,ttm,12/30/2020,12/30/2019,12/30/2018,12/30/2017
0,Total Revenue,"34,734,000","32,184,000","32,136,000","32,765,000","31,657,000"
1,Operating Revenue,"34,734,000","32,184,000","32,136,000","32,765,000","31,657,000"
2,Cost of Revenue,"17,935,000","16,605,000","17,136,000","16,682,000","16,001,000"
3,Gross Profit,"16,799,000","15,579,000","15,000,000","16,083,000","15,656,000"
4,Operating Expense,"8,943,000","8,757,000","8,872,000","9,350,000","8,422,000"
5,Selling General and Administrative,"6,988,000","6,879,000","6,961,000","7,529,000","6,572,000"
6,General & Administrative Expense,"6,988,000","6,879,000","6,961,000","7,529,000","6,572,000"
7,Salaries and Wages,"-133,000","-50,000","-68,000","-73,000",-
8,Other G and A,"7,121,000","6,929,000","7,029,000","7,602,000","6,572,000"
9,Research & Development,"1,955,000","1,878,000","1,911,000","1,821,000","1,850,000"


In [14]:
Financial_Statement          = yahoo_financial_statement(ticker='MMM',selenium_path=r'/Users/chen-lichiang/Desktop/python/Factor_Analysis/chromedriver',fetch_data='Balance_Sheet')
Company_Financial_Statement  = Financial_Statement.Web_Driver()
Company_Financial_Statement  = Financial_Statement.get_table_data()
    

Company_Financial_Statement

,Breakdown,12/30/2020,12/30/2019,12/30/2018,12/30/2017
0,Total Assets,"47,344,000","44,659,000","36,500,000","37,987,000"
1,Current Assets,"14,982,000","12,971,000","13,709,000","14,277,000"
2,"Cash, Cash Equivalents & Short Term Investments","5,038,000","2,451,000","3,233,000","4,129,000"
3,Cash And Cash Equivalents,"4,634,000","2,353,000","2,853,000","3,053,000"
4,Other Short Term Investments,"404,000","98,000","380,000","1,076,000"
...,...,...,...,...,...
84,Total Debt,"19,753,000","21,278,000","14,697,000","14,009,000"
85,Net Debt,"14,161,000","17,960,000","11,769,000","10,896,000"
86,Share Issued,"944,033","944,033","944,033","944,033"
87,Ordinary Shares Number,"577,750","575,185","576,575","594,884"


In [16]:
Financial_Statement          = yahoo_financial_statement(ticker='MMM',selenium_path=r'/Users/chen-lichiang/Desktop/python/Factor_Analysis/chromedriver',fetch_data='Cash_Flow')
Company_Financial_Statement  = Financial_Statement.Web_Driver()
Company_Financial_Statement  = Financial_Statement.get_table_data()
    

Company_Financial_Statement

,Breakdown,ttm,12/30/2020,12/30/2019,12/30/2018,12/30/2017
0,Operating Cash Flow,"8,570,000","8,113,000","7,070,000","6,439,000","6,240,000"
1,Cash Flow from Continuing Operating Activities,"8,570,000","8,113,000","7,070,000","6,439,000","6,240,000"
2,Net Income from Continuing Operations,"5,927,000","5,388,000","4,582,000","5,363,000","4,869,000"
3,Operating Gains Losses,"343,000","17,000","408,000","-135,000","-253,000"
4,Gain Loss On Sale of Business,0,"-389,000","51,000","-545,000","-586,000"
5,Pension And Employee Benefit Expense,"343,000","406,000","357,000","410,000","333,000"
6,Depreciation Amortization Depletion,"1,911,000","1,911,000","1,593,000","1,488,000","1,544,000"
7,Depreciation & amortization,"1,911,000","1,911,000","1,593,000","1,488,000","1,544,000"
8,Deferred Tax,"-115,000","-165,000","-273,000","-57,000","107,000"
9,Deferred Income Tax,"-115,000","-165,000","-273,000","-57,000","107,000"
